In [ ]:
function rosenbrock(x::AbstractVector)
    return sum(100 .* (x[2:end] .- x[1:end-1].^2).^2 .+ (1 .- x[1:end-1]).^2)
end


function rosenbrock_gradient(x::AbstractVector)
    n = length(x)
    grad = zeros(n)
    grad[1] = -400 * x[1] * (x[2] - x[1]^2) + 2 * (x[1] - 1)
    for i in 2:n-1
        grad[i] = 200 * (x[i] - x[i-1]^2) - 400 * x[i] * (x[i+1] - x[i]^2) + 2 * (x[i] - 1)
    end
    grad[n] = 200 * (x[n] - x[n-1]^2)
    return grad
end


function rosenbrock_hessian(x::AbstractVector)
    n = length(x)
    hessian = zeros(n, n)
    hessian[1, 1] = 1200*x[1]^2 - 400*x[2] + 2
    for i in 1:n-1
        hessian[i, i+1] = -400*x[i]
        hessian[i+1, i] = -400*x[i]
    end
    for i in 2:n-1
        hessian[i, i] = 202 + 1200*x[i]^2 - 400*x[i+1]
    end
    hessian[n, n] = 200
    return hessian
end

In [ ]:
rng = MersenneTwister(1234)

function generate_random_gaussian_matrix(s::Int, n::Int)
    P = randn(rng, Float64, (s, n))
    return P ./ sqrt(s)
end



function solve_subproblem(g::Array{Float64}, H::Array{Float64}, delta::Float64, n::Int)
    F = [H g; g' -delta]
    vals, vecs, info = KrylovKit.eigsolve(
            F, n + 1, 1, :SR, Float64;
            issymmetric=true, tol=1e-12
        )
    return vecs[1]
end


function calc_stepsize_with_ls(linesearch, loss, der, x, d)
    φ(eta) = loss(x .+ eta .* d)
    function dφ(eta)
        return dot(der(x .+ eta .* d), d)
    end
    function φdφ(eta)
        return (φ(eta), dφ(eta))
    end

    fx = loss(x)
    gx = der(x)
    dφ0 = dot(d, gx)
    try
        eta, fx = linesearch(φ, dφ, φdφ, 1.0, fx, dφ0)
        return eta
    catch e
        println("Line search failed: ", e)
        return 1.0
    end
end

In [ ]:
# HSODM
function HSODM_linesearch(
    x::Array{Float64},
    loss::Function,
    der::Function,
    hes::Function,
    nu::Float64,
    Delta::Float64,
    delta::Float64,
    n::Int,
    maxiter::Int,
    eps_grad::Float64,
    linesearch=LineSearches.HagerZhang()
)
    xs = []
    timestamps = []
    obj_vals = []
    grad_norms = []
    elapsed_time = 0.0
    terminate = false

    push!(xs, x)
    push!(timestamps, elapsed_time)
    push!(obj_vals, loss(x))
    push!(grad_norms, norm(der(x)))

    @showprogress dt=1 desc="HSODM_linesearch" for i in 1:maxiter
        runtime = @elapsed begin
            g::Array{Float64} = der(x)
            H::Array{Float64} = Symmetric(hes(x))
            subprob_sol::Array{Float64} = solve_subproblem(g, H, delta, n)
            v::Array{Float64} = subprob_sol[begin:end-1]
            t::Float64 = subprob_sol[end]

            if abs(t) >= nu
                d = v ./ t
            else
                sign = ifelse(- dot(g, v) > 0, 1, -1)
                d = sign .* v
            end

            norm_d = norm(d)
            if norm_d > Delta
                eta = calc_stepsize_with_ls(linesearch, loss, der, x, d)
                x = x .+ eta .* d
            else
                x += d
                println("norm_d <= Delta")
                terminate = true
            end
        end

        elapsed_time += runtime

        push!(xs, x)
        push!(timestamps, elapsed_time)
        push!(obj_vals, loss(x))
        push!(grad_norms, norm(der(x)))

        if grad_norms[end] < eps_grad
            println("grad_norms[end] < eps_grad")
            terminate = true
        end

        if terminate
            break
        end
    end

    println("Norm of gradient: \t\t\t\t\t", norm(der(x)))

    return xs, timestamps, obj_vals, grad_norms
end



In [ ]:
function RSTRM_linesearch(
    x::Array{Float64, 1},
    loss::Function,
    der::Function,
    hes::Function,
    nu::Float64,
    Delta::Float64,
    delta::Float64,
    n::Int,
    s::Int,
    max_iter::Int,
    eps_grad::Float64,
    linesearch=LineSearches.BackTracking()
)
    xs = []
    timestamps = []
    obj_vals = []
    grad_norms = []
    count = 0
    iter = 0
    terminate = false
    elapsed_time = 0.0

    push!(xs, x)
    push!(timestamps, elapsed_time)
    push!(obj_vals, loss(x))
    push!(grad_norms, norm(der(x)))


    @showprogress dt=1 desc="RSTRM_linesearch (s=$s)" for i in 1:max_iter
        runtime = @elapsed begin
            g::Array{Float64} = der(x)
            H::Array{Float64} = Symmetric(hes(x))
            P::Array{Float64} = generate_random_gaussian_matrix(s, n)
            PT::Array{Float64} = P'
            g_sub = P * g
            H_sub = P * H * PT

            subprob_sol::Array{Float64} = solve_subproblem(g_sub, H_sub, delta, s)
            v_sub::Array{Float64} = subprob_sol[begin:end-1]
            t::Float64 = subprob_sol[end]

            if abs(t) > nu
                d = PT * v_sub ./ t
            else
                sign = ifelse(- dot(g_sub, v_sub) > 0, 1, -1)
                d = sign .* PT * v_sub
            end

            norm_d = norm(d)
            if norm_d > Delta
                eta = calc_stepsize_with_ls(linesearch, loss, der, x, d)
                x = x .+ eta .* d
            else
                x += d
                println("norm_d <= Delta")
                terminate = true
            end
        end
        elapsed_time += runtime

        push!(xs, x)
        push!(timestamps, elapsed_time)
        push!(obj_vals, loss(x))
        push!(grad_norms, norm(der(x)))

        if grad_norms[end] < eps_grad
            println("grad_norm[end] < eps_grad")
            terminate = true
        end

        if terminate
            break
        end
    end

    println("Norm of gradient: ", norm(der(x)))

    return xs, timestamps, obj_vals, grad_norms
end


In [ ]:
function GD_linesearch(
    x::Array{Float64},
    loss::Function,
    der::Function,
    nu::Float64,
    maxiter::Int,
    eps_grad::Float64,
    linesearch=LineSearches.HagerZhang()
)
    xs = []
    timestamps = []
    obj_vals = []
    grad_norms = []
    elapsed_time = 0.0
    terminate = false

    push!(xs, x)
    push!(timestamps, elapsed_time)
    push!(obj_vals, loss(x))
    push!(grad_norms, norm(der(x)))

    @showprogress dt=1 desc="GD_linesearch" for i in 1:maxiter
        runtime = @elapsed begin
            g::Array{Float64} = der(x)
            d = -g
            eta = calc_stepsize_with_ls(linesearch, loss, der, x, d)
            x = x .+ eta .* d
        end

        elapsed_time += runtime

        push!(xs, x)
        push!(timestamps, elapsed_time)
        push!(obj_vals, loss(x))
        push!(grad_norms, norm(der(x)))

        if grad_norms[end] < eps_grad
            terminate = true
        end

        if terminate
            break
        end
    end

    println("Norm of gradient: \t\t\t\t\t", norm(der(x)))

    return xs, timestamps, obj_vals, grad_norms
end

In [ ]:
n = 1000
r = 25
s_list = [25, 50, 75]
nu = 0.1
eps = 1e-8
Delta = eps^(1/2) 
delta = eps^(1/2)  # * (1 + xi_1)^2 * n / s in RSTRM
max_iter = 20000  # * 2 in GD
eps_grad = 0.0

Random.seed!(1234)
x0 = randn(n)
x_opt = ones(n)
R = generate_random_gaussian_matrix(r, n)
loss(x) = rosenbrock_rank_deficient(x, R)
der(x) = rosenbrock_rank_deficient_gradient(x, R)
hes(x) = rosenbrock_rank_deficient_hessian(x, R)


# HSODM
xs, timestamps, obj_vals, grad_norms = HSODM_linesearch(
    copy(x0), loss, der, hes,
    nu, Delta, delta, n, max_iter, eps_grad
)
plot_obj_vals_vs_iter = plot(
    obj_vals,
    xlabel="Iteration",
    ylabel="Objective function value",
    lw=2,
    yscale=:log10,
    label="HSODM (n=$n)"
)
plot_obj_vals_vs_time = plot(
    timestamps,
    obj_vals,
    xlabel="Time (s)",
    ylabel="Objective function value",
    lw=2,
    yscale=:log10,
    label="HSODM (n=$n)"
)
plot_grad_norms_vs_iter = plot(
    grad_norms,
    xlabel="Iteration",
    ylabel="Norm of gradient",
    lw=2,
    yscale=:log10,
    label="HSODM (n=$n)"
)
plot_grad_norms_vs_time = plot(
    timestamps,
    grad_norms,
    xlabel="Time (s)",
    ylabel="Norm of gradient",
    lw=2,
    yscale=:log10,
    label="HSODM (n=$n)"
)


# GD
xs, timestamps, obj_vals, grad_norms = GD_linesearch(
    copy(x0), loss, der, nu, 2 * max_iter, eps_grad
)
plot!(
    plot_obj_vals_vs_iter,
    obj_vals,
    lw=2,
    yscale=:log10,
    label="GD (n=$n)"
)
plot!(
    plot_obj_vals_vs_time,
    timestamps,
    obj_vals,
    lw=2,
    yscale=:log10,
    label="GD (n=$n)"
)
plot!(
    plot_grad_norms_vs_iter,
    grad_norms,
    lw=2,
    yscale=:log10,
    label="GD (n=$n)"
)
plot!(
    plot_grad_norms_vs_time,
    timestamps,
    grad_norms,
    lw=2,
    yscale=:log10,
    label="GD (n=$n)"
)


# RSTRM
for s in s_list
    xi_1 = sqrt(s/n)
    xs, timestamps, obj_vals, grad_norms = RSTRM_linesearch(
        copy(x0), loss, der, hes,
        nu, Delta, delta * ((1 + xi_1)^2) * n / s, n, s, max_iter, eps_grad
    )
    plot!(
        plot_obj_vals_vs_iter,
        obj_vals,
        lw=2,
        yscale=:log10,
        label="RSTRM (s=$s)"
    )
    plot!(
        plot_obj_vals_vs_time,
        timestamps,
        obj_vals,
        lw=2,
        yscale=:log10,
        label="RSTRM (s=$s)"
    )
    plot!(
        plot_grad_norms_vs_iter,
        grad_norms,
        lw=2,
        yscale=:log10,
        label="RSTRM (s=$s)"
    )
    plot!(
        plot_grad_norms_vs_time,
        timestamps,
        grad_norms,
        lw=2,
        yscale=:log10,
        label="RSTRM (s=$s)"
    )
end


In [ ]:
plot!(plot_obj_vals_vs_iter, xlim=(0,100), ylim=(1e-8, 1e8), legendfontsize=10)
display(plot_obj_vals_vs_iter)

plot!(plot_obj_vals_vs_time, xlim=(0,5), ylim=(1e-8,1e8), legendfontsize=10)
display(plot_obj_vals_vs_time)

plot!(plot_grad_norms_vs_iter, xlim=(0,100), ylim=(1e-8, 1e8), legendfontsize=10)
display(plot_grad_norms_vs_iter)

plot!(plot_grad_norms_vs_time, xlim=(0,5), ylim=(1e-8,1e8), legendfontsize=10)
display(plot_grad_norms_vs_time)